In [16]:
import os
import glob
import xml.etree.ElementTree as ET
import cPickle as pickle

In [21]:
xml_directory = '../Wikipedia/xml/'
instances = {}
for filename in glob.glob(os.path.join(xml_directory, '*.xml')):
    with open(filename, 'r') as xml_file:
        instance = xml_file.read().decode('utf-8', 'ignore')
        try:
            tempTitle = filename[:-4].replace('../Wikipedia/xml/', '')
            root = ET.fromstring(instance)
            instances[tempTitle] = {'sentence': [], 'annotated_sentence': [], 'triples': []}
            for i in range(0, len(root)):
                instances[tempTitle]['sentence'].append(root[i][0].text.replace('\t', '').decode('utf-8', 'ignore'))
                instances[tempTitle]['annotated_sentence'].append(root[i][2].text.replace('\t', '').decode('utf-8', 'ignore'))
                instances[tempTitle]['triples'].append([])
                for j in range(0, len(root[i][5])):
                    instances[tempTitle]['triples'][len(instances[tempTitle]['triples']) - 1].append(root[i][5][j].text.replace('\n', '').\
                                                                               replace('\t', '').\
                                                                               decode('utf-8', 'ignore'))
        except ET.ParseError:
            pass

In [22]:
len(instances['Alan_Bean']['annotated_sentence'])

54

In [23]:
instances['Alan_Bean']['triples'][1]

[u'http://dbpedia.org/resource/Alan_Bean http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/ontology/Astronaut',
 u'http://dbpedia.org/resource/Alan_Bean http://dbpedia.org/ontology/type http://dbpedia.org/resource/NASA']

In [24]:
dataset_location = '../Wikipedia/Dataset.p'
with open(dataset_location, 'wb') as f:
    pickle.dump(instances, f)